In [1]:
import projectpath

from importlib import resources
import ipywidgets as widgets
import jax
import json
import matplotlib.pyplot as plt
import numpy as np
import time

from mosmo.model import Molecule, Reaction, Pathway, ReactionNetwork
from mosmo.knowledge import kb
from mosmo.calc import fba_gd
from mosmo.preso.escher import escher_map
import mosmo.preso.escher.pw as pw_files

# Setup matplotlib to play nice with widgets
%matplotlib widget
plt.ioff()

# Setup jax to use float64, and get the No GPU warning out of the way
jax.config.update('jax_enable_x64', True)
prng = jax.random.PRNGKey(int(time.time() * 1000))

# Setup the KB
KB = kb.configure_kb()

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


## Glycolysis, Pentose Phosphate, and Entner-Doudoroff

In [2]:
network = ReactionNetwork()
for pw_name in ['glycolysis', 'pentose phosphate', 'entner-doudoroff']:
    pws = KB.find(KB.pathways, pw_name)
    if not pws:
        raise ValueError(f'{pw_name} not found in {KB.pathways}')
    if len(pws) > 1:
        print(f'Multiple hits to "{pw_name}"')
    for pw in pws:
        for step in pw.steps:
            network.add_reaction(step)

boundaries = [KB(met_id) for met_id in [
    "Glc.D.6P",
    "accoa",
    "coa",
    "amp",
    "adp",
    "atp",
    "pi",
    "nad.ox",
    "nad.red",
    "nadp.ox",
    "nadp.red",
    "co2",
    "h+",
    "h2o",
]]
intermediates = [met for met in network.reactants if met not in boundaries]

print(f'Network has {network.shape[0]} reactants ({len(intermediates)} intermediates + {len(boundaries)} boundaries) in {network.shape[1]} reactions')

Network has 31 reactants (17 intermediates + 14 boundaries) in 22 reactions


## FBA balancing demand for acCoA, ATP, and NADPH

In [5]:
class InteractiveFba:
    def __init__(self, network, boundaries, targets, map_json):
        self.network = network
        self.boundaries = boundaries

        # Set up the FBA problem
        intermediates = [met for met in network.reactants if met not in boundaries]
        self.fba = fba_gd.FbaGd(network, intermediates, {"targets": fba_gd.ProductionObjective(network, targets)})
        
        # Boundary flux plot
        fig, ax = plt.subplots(figsize=(3, 4))
        fig.canvas.header_visible = False
        fig.canvas.footer_visible = False
        fig.canvas.toolbar_visible = False
        fig.canvas.resizable = False

        y = np.arange(len(boundaries))
        ax.set_title("Boundary Fluxes")
        ax.set_xlabel('Net Flux')
        ax.set_xlim(-10, 10)
        ax.set_yticks(y, labels=[b.label for b in boundaries])
        ax.invert_yaxis()
        ax.grid(True)
        ax.axvline(0)
        fig.tight_layout()
        
        self.boundary_plot = fig
        self.bars = ax.barh(y, width=0)
        self.boundary_indices = np.array([network.reactants.index_of(b) for b in boundaries])

        # Pathway diagram to show reaction fluxes
        self.diagram = escher_map.EscherMap(
            map_json,
            width="12cm",
            reaction_scale=escher_map.Scale({0: ("#eeeeee", 5), 5: ("#1f77b4", 40)}, use_abs=True),
        )
        # Draw everything with the initial solution before displaying it
        self.show_results(self.fba.solve())

        # Set up the controls and update logic
        self.control_map = {}
        for target, value in targets.items():
            control = widgets.FloatSlider(
                value=value,
                description=target.label,
                min=0,
                max=5.0,
                step=0.1,
                continuous_update=False,
                readout=True,
                readout_format=".1f")
            control.observe(self.update, names="value")
            self.control_map[control] = target

        # Finally, lay it all out in a dashboard
        self.dashboard = widgets.HBox([
            widgets.VBox([
                widgets.VBox(list(self.control_map.keys())),
                fig.canvas,
            ], layout=widgets.Layout(width="40%")),
            self.diagram.widget,
        ], layout = widgets.Layout(width='1000px', height='600px', border='1px solid green'))

    def show_results(self, soln):
        # Update the Boundary Fluxes plot
        for bar, value in zip(self.bars, soln.dmdt[self.boundary_indices]):
            bar.set_width(value)
        self.boundary_plot.canvas.draw_idle()

        # Update the pathway diagram
        self.diagram.draw(reaction_data={rxn.label: flux for rxn, flux in network.reactions.unpack(soln.velocities).items()})

    def update(self, change):
        if change.type == "change":
            self.fba.update_params({"targets": {self.control_map[change.owner]: change.new}})
            self.show_results(self.fba.solve())


In [6]:
ifba = InteractiveFba(
    network,
    boundaries,
    {KB('accoa'): 2, KB('atp'): 3, KB('nadp.red'): 0},
    json.loads(resources.read_text(pw_files, "glycolysis_ppp_ed.json"))
)
ifba.dashboard

In [7]:
ifba2 = InteractiveFba(
    network,
    boundaries + [KB("Rib.D.5P"), KB("Ery.D.4P")],
    {KB('accoa'): 2, KB("Rib.D.5P"): 0, KB("Ery.D.4P"): 0},
    json.loads(resources.read_text(pw_files, "glycolysis_ppp_ed.json"))
)
ifba2.dashboard